In [1]:
### Imports
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests

In [2]:
### Website
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://www.jfkairport.com/at-airport/shops-restaurants-and-services')

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:01<00:00, 6.22MB/s]


In [92]:
### All Info
all = list(map(lambda name: name.text, driver.find_elements_by_class_name('tiles-wrapper')))
data = pd.DataFrame(all)
data.rename(columns={0:'Name'}, inplace=True)
jfk = data["Name"].str.split("\n", n = 3, expand = True)
jfk.rename(columns={0:'Name',1:'Location',2:'Description1'}, inplace=True)

### Clean Description Column 
jfk['Description'] = jfk['Description1'].str.extract('(.+?),')
jfk['Description'] = jfk['Description'].fillna(jfk['Description1'])
jfk.drop(['Description1'], axis = 1, inplace = True) 

### Final DF with only restaurants 
condition = ['Grab & Go', 'Restaurants', '$2 Water']
jfk_final = jfk[(jfk['Description'].isin(condition))]
jfk_final.reset_index(drop=True, inplace=True)
jfk_final.head()

,Name,Location,Description
0,Air Bar / Tim Hortons / NY Deli,Jamaica Station AirTrain Station,Grab & Go
1,Arrivals Eatery,Pre-Security,$2 Water
2,Downtown Market,Post Security,$2 Water
3,Dunkin Donuts Mezz Level,Pre-Security,$2 Water
4,Dunkin' Donuts (kiosk),Food Court,Grab & Go


In [88]:
### Shop Name
name = list(map(lambda name: name.text, driver.find_elements_by_css_selector('h2')))
df_name =  pd.DataFrame(name)
df_name.rename(columns={0:'Name'}, inplace=True)

### Description
type = list(map(lambda name: name.text, driver.find_elements_by_css_selector('small')))
description = type[5:]
df_description =  pd.DataFrame(description)
df_description.rename(columns={0:'Description1'}, inplace=True)

df1 = pd.concat([df_name, df_description], axis =1)

### Clean Description Column 
df1['Description'] = df1['Description1'].str.extract('(.+?),')
df1['Description'] = df1['Description'].fillna(df1['Description1'])
df1.drop(['Description1'], axis = 1, inplace = True) 

### Final DF with only restaurants 
condition = ['Grab & Go', 'Restaurants', '$2 Water']
jfk_rest = df1[(df1['Description'].isin(condition))]
jfk_rest.reset_index(drop=True, inplace=True)
jfk_rest.head()

,Name,Description
0,Air Bar / Tim Hortons / NY Deli,Grab & Go
1,Arrivals Eatery,$2 Water
2,Downtown Market,$2 Water
3,Dunkin Donuts Mezz Level,$2 Water
4,Dunkin' Donuts (kiosk),Grab & Go


In [96]:
jfk_rest['Name'].sort_values()

99                    5ive Steak
137                       APT 7B
148              AbitiNo's Pizza
149              AbitiNo's Pizza
150               Abitinos Pizza
                 ...            
38                    Wok & Roll
39                    Wok & Roll
58      World Bean Baggage Claim
59     World Bean Gate 65 (Main)
120                 iS Duty Free
Name: Name, Length: 176, dtype: object